In [1]:
import numpy as np
import pandas as pd

from django.conf import settings
from django.db import connections

import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [9]:
sql='''
SELECT
DATE(DATE_ADD) DATE_ADD,
YEAR(DATE_ADD) Y,
QUARTER(DATE_ADD) q,
MONTH(DATE_ADD) m,
DAY(DATE_ADD) d,
SUM((SELECT SUM(product_quantity) FROM ps17_order_detail d WHERE id_order=o.id_order)) products,
SUM((SELECT SUM(product_quantity) FROM ps17_order_detail d WHERE id_order=o.id_order AND reduction_percent>0)) products_discounted,
COUNT(id_order) orders,
ROUND((
SUM((SELECT SUM(original_wholesale_price) FROM ps17_order_detail d WHERE id_order=o.id_order)) 
    ),2) GBP_cost
,
ROUND(SUM((total_products_wt-total_discounts_tax_incl)/conversion_rate) ,2) GBP_products,

ROUND(SUM((total_shipping_tax_incl)/conversion_rate) ,2) GBP_shipping,
ROUND(SUM((total_paid_real)/conversion_rate) ,2) GBP_paid


FROM `ps17_orders` o
WHERE current_state IN
(SELECT id_order_state FROM ps17_order_state WHERE paid=1)

GROUP BY Y,m,d
ORDER BY Y ,m,d
'''

sql


'\nSELECT\nDATE(DATE_ADD) DATE_ADD,\nYEAR(DATE_ADD) Y,\nQUARTER(DATE_ADD) q,\nMONTH(DATE_ADD) m,\nDAY(DATE_ADD) d,\nSUM((SELECT SUM(product_quantity) FROM ps17_order_detail d WHERE id_order=o.id_order)) products,\nSUM((SELECT SUM(product_quantity) FROM ps17_order_detail d WHERE id_order=o.id_order AND reduction_percent>0)) products_discounted,\nCOUNT(id_order) orders,\nROUND((\nSUM((SELECT SUM(original_wholesale_price) FROM ps17_order_detail d WHERE id_order=o.id_order)) \n    ),2) GBP_cost\n,\nROUND(SUM((total_products_wt-total_discounts_tax_incl)/conversion_rate) ,2) GBP_products,\n\nROUND(SUM((total_shipping_tax_incl)/conversion_rate) ,2) GBP_shipping,\nROUND(SUM((total_paid_real)/conversion_rate) ,2) GBP_paid\n\n\nFROM `ps17_orders` o\nWHERE current_state IN\n(SELECT id_order_state FROM ps17_order_state WHERE paid=1)\n\nGROUP BY Y,m,d\nORDER BY Y ,m,d\n'

In [3]:
cols= ['DT','Y','Q','M','D','products','products_discounted','orders','GBP_cost','GBP_products','GBP_shipping','GBP_paid']
cols

['DT',
 'Y',
 'Q',
 'M',
 'D',
 'products',
 'products_discounted',
 'orders',
 'GBP_cost',
 'GBP_products',
 'GBP_shipping',
 'GBP_paid']

In [10]:
def get_data():
    with connections['presta'].cursor() as cursor:
        cursor.execute(sql,)
        rows = cursor.fetchall()

    return rows    

In [21]:
p = pd.DataFrame(get_data(), columns=cols)
#p=p.sort_index(ascending=False,inplace=False)
p

,DT,Y,Q,M,D,products,products_discounted,orders,GBP_cost,GBP_products,GBP_shipping,GBP_paid
0,2018-10-18,2018,4,10,18,33,None,2,9.24,156.18,7.00,326.37
1,2018-10-19,2018,4,10,19,12,1,3,9.24,77.58,10.50,119.25
2,2018-10-20,2018,4,10,20,4,None,2,5.28,30.14,12.00,42.14
3,2018-10-23,2018,4,10,23,5,None,1,5.28,26.34,3.50,29.84
4,2018-10-26,2018,4,10,26,3,None,1,0.00,11.65,3.50,15.15
...,...,...,...,...,...,...,...,...,...,...,...,...
937,2021-12-01,2021,4,12,1,17,None,3,19.66,120.29,20.85,141.14
938,2021-12-02,2021,4,12,2,6,None,1,8.43,22.73,6.95,29.68
939,2021-12-03,2021,4,12,3,12,None,1,21.44,95.35,6.95,102.30
940,2021-12-04,2021,4,12,4,9,None,2,11.24,67.01,13.90,80.91


In [22]:
for i in range(6,32):
    p=p.append({'DT': f'2021-12-{i}','Y': 2021,'Q':4,'M':12,'D':i},ignore_index=True)

p

,DT,Y,Q,M,D,products,products_discounted,orders,GBP_cost,GBP_products,GBP_shipping,GBP_paid
0,2018-10-18,2018,4,10,18,33,None,2.0,9.24,156.18,7.00,326.37
1,2018-10-19,2018,4,10,19,12,1,3.0,9.24,77.58,10.50,119.25
2,2018-10-20,2018,4,10,20,4,None,2.0,5.28,30.14,12.00,42.14
3,2018-10-23,2018,4,10,23,5,None,1.0,5.28,26.34,3.50,29.84
4,2018-10-26,2018,4,10,26,3,None,1.0,0.00,11.65,3.50,15.15
...,...,...,...,...,...,...,...,...,...,...,...,...
963,2021-12-27,2021,4,12,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN
964,2021-12-28,2021,4,12,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN
965,2021-12-29,2021,4,12,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN
966,2021-12-30,2021,4,12,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
p[-30:]

,DT,Y,Q,M,D,products,products_discounted,orders,GBP_cost,GBP_products,GBP_shipping,GBP_paid
938,2021-12-02,2021,4,12,2,6,None,1.0,8.43,22.73,6.95,29.68
939,2021-12-03,2021,4,12,3,12,None,1.0,21.44,95.35,6.95,102.30
940,2021-12-04,2021,4,12,4,9,None,2.0,11.24,67.01,13.90,80.91
941,2021-12-05,2021,4,12,5,5,None,1.0,8.62,58.03,6.95,64.98
942,2021-12-6,2021,4,12,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
943,2021-12-7,2021,4,12,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
944,2021-12-8,2021,4,12,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
945,2021-12-9,2021,4,12,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
946,2021-12-10,2021,4,12,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
947,2021-12-11,2021,4,12,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
p.interpolate(method='linear', limit_direction='forward', axis=0)

,DT,Y,Q,M,D,products,products_discounted,orders,GBP_cost,GBP_products,GBP_shipping,GBP_paid
0,2018-10-18,2018,4,10,18,33,None,2.0,9.24,156.18,7.00,326.37
1,2018-10-19,2018,4,10,19,12,1,3.0,9.24,77.58,10.50,119.25
2,2018-10-20,2018,4,10,20,4,None,2.0,5.28,30.14,12.00,42.14
3,2018-10-23,2018,4,10,23,5,None,1.0,5.28,26.34,3.50,29.84
4,2018-10-26,2018,4,10,26,3,None,1.0,0.00,11.65,3.50,15.15
...,...,...,...,...,...,...,...,...,...,...,...,...
963,2021-12-27,2021,4,12,27,NaN,NaN,1.0,NaN,NaN,NaN,NaN
964,2021-12-28,2021,4,12,28,NaN,NaN,1.0,NaN,NaN,NaN,NaN
965,2021-12-29,2021,4,12,29,NaN,NaN,1.0,NaN,NaN,NaN,NaN
966,2021-12-30,2021,4,12,30,NaN,NaN,1.0,NaN,NaN,NaN,NaN


In [7]:
p['GBP_shipping'] = p['GBP_shipping'].astype(float)
p['GBP_products'] = p['GBP_products'].astype(float)
p['GBP_cost'] = p['GBP_cost'].astype(float)

In [8]:
p['VAT8pc'] = round(p['GBP_products']*0.08,2)

In [9]:
p['GrossM'] = p['GBP_products'].astype(float) - p['VAT8pc'] - p['GBP_cost'].astype(float)

In [13]:
p['Y-Q'] = p['Y'].astype(str)+'-'+p['Q'].astype(str)

In [14]:
p.reset_index(inplace=True)
p

,level_0,index,Y,Q,M,D,products,products_discounted,orders,GBP_cost,GBP_products,GBP_shipping,GBP_paid,VAT8pc,GrossM,Y-Q
0,0,0,2021,4,12,4,4,None,1,4.13,32.39,6.95,39.34,2.59,25.67,2021-4
1,1,1,2021,4,12,3,12,None,1,21.44,95.35,6.95,102.30,7.63,66.28,2021-4
2,2,2,2021,4,12,2,6,None,1,8.43,22.73,6.95,29.68,1.82,12.48,2021-4
3,3,3,2021,4,12,1,17,None,3,19.66,120.29,20.85,141.14,9.62,91.01,2021-4
4,4,4,2021,4,11,30,6,None,2,15.80,90.39,13.90,104.29,7.23,67.36,2021-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
936,936,936,2018,4,10,26,3,None,1,0.00,11.65,3.50,15.15,0.93,10.72,2018-4
937,937,937,2018,4,10,23,5,None,1,5.28,26.34,3.50,29.84,2.11,18.95,2018-4
938,938,938,2018,4,10,20,4,None,2,5.28,30.14,12.00,42.14,2.41,22.45,2018-4
939,939,939,2018,4,10,19,12,1,3,9.24,77.58,10.50,119.25,6.21,62.13,2018-4
